In [7]:
import os
import re
import subprocess

import spiceypy as spice

import ale

from ale import util

from ale.base.data_naif import NaifSpice
from ale.base.label_isis import IsisLabel
from ale.base.type_sensor import Framer
from ale.base.base import Driver

from ale.formatters.isis_formatter import to_isis

In [2]:
ale_root = os.path.split(ale.__file__)[0]
data_root = os.path.join(ale_root, '../tests/pytests/data')
dirs = next(os.walk(data_root, topdown=True))[1]
dirs = [d for d in dirs if not d.startswith('.')]
image_2_data = {}

for d in dirs:
    tmp = os.path.join(data_root, d)
    image_2_data[d] = [os.path.join(tmp, f) for f in os.listdir(tmp) if not f.startswith('.') and os.path.splitext(f)[1] != '.lbl']

def get_image_label(image, label_type='pds3'):
    if not isinstance(image, str):
        try:
            image = str(image)
        except:
            raise KeyError('Cannot coerce requested image name to string')

    label_file = glob(os.path.join(data_root, '*',f'{image}_{label_type}.lbl'))
    if not label_file:
        raise Exception(f'Could not find label file for {image}')

    return label_file[0]

def get_image_kernels(image):
    """
    Get the kernels to use with a test image.

    Parameters
    ----------
    image : str
            The image name to get kernels for. I.E. 'EN1072174528M'

    Returns
    -------
    : list
      A list containing the absolute paths to the kernels for the images.
      This list contains all of the kernel files available in the test image's
      data directory which may contain transfer files that need to be converted
      to binary files.
    """
    if not isinstance(image, str):
        try:
            image = str(image)
        except:
            raise KeyError('Cannot coerce requested image name to string')
    if image in image_2_data:
        return image_2_data[image]
    else:
        raise KeyError('Could not find test data for' + image)

def convert_kernels(kernels):
    """
    Convert any transfer kernels in a list to binary kernels

    Parameters
    ----------
    kernels : list
              A list of kernels. Only transfer kernels present in the list will
              be converted. Non-transfer kernels will be ignored.

    Returns
    -------
    updated_kernels : list
                      The updated kernel list where all transfer kernels have
                      been replaced with their converted binary kernel. This
                      is designed to be passed directly to furnsh.
    binary_kernels : list
                     The list of binary kernels created.
    """
    binary_kernels = []
    updated_kernels = []
    for kernel in kernels:
        split_kernel = os.path.splitext(kernel)
        if 'x' in split_kernel[1].lower():
            # Get the full path to the kernel then truncate it to the relative path
            path = os.path.join(data_root, kernel)
            path = os.path.relpath(path)
            bin_output = subprocess.run(['tobin', path],
                                        capture_output=True, check=True)
            matches = re.search(r'To: (.*\.b\w*)', str(bin_output.stdout))
            if not matches:
                warnings.warn('Failed to convert transfer kernel, ' + kernel + ', skipping...')
            else:
                kernel = matches.group(1)
                binary_kernels.append(kernel)
        updated_kernels.append(kernel)
    return updated_kernels, binary_kernels

In [3]:
file = '/home/acpaquette/repos/ale/tests/pytests/data/JNCR_2016240_01M06152_V01/JNCR_2016240_01M06152_V01_METHANE_0008_isis.lbl'
kernels = get_image_kernels('JNCR_2016240_01M06152_V01')
updated_kernels, binary_kernels = convert_kernels(kernels)
updated_kernels

['../tests/pytests/data/JNCR_2016240_01M06152_V01/ab103105.spice_0.bsp',
 '../tests/pytests/data/JNCR_2016240_01M06152_V01/ab103105.spice_1.bsp',
 '../tests/pytests/data/JNCR_2016240_01M06152_V01/JNCR_2016240_01M06152_V01_METHANE_0008_0.bsp',
 '../tests/pytests/data/JNCR_2016240_01M06152_V01/JNCR_2016240_01M06152_V01_METHANE_0008_1.bsp',
 '../tests/pytests/data/JNCR_2016240_01M06152_V01/juno_sc_rec_160821_160827_v01_0_sliced_-61000.bc',
 '../tests/pytests/data/JNCR_2016240_01M06152_V01/juno_sc_rec_160821_160827_v01_1_sliced_-61000.bc',
 '../tests/pytests/data/JNCR_2016240_01M06152_V01/mgs_sc_ab1_0_sliced_-94000.bc',
 '../tests/pytests/data/JNCR_2016240_01M06152_V01/mgs_sc_ab1_1_sliced_-94000.bc',
 '/home/acpaquette/repos/ale/ale/../tests/pytests/data/JNCR_2016240_01M06152_V01/print.prt',
 '/home/acpaquette/repos/ale/ale/../tests/pytests/data/JNCR_2016240_01M06152_V01/naif0012.tls',
 '/home/acpaquette/repos/ale/ale/../tests/pytests/data/JNCR_2016240_01M06152_V01/pck00010.tpc',
 '/home/a

In [35]:
class juno_test(Framer, IsisLabel, NaifSpice, Driver):
    
    @property
    def instrument_id(self):
        look_up = {'JNC': 'JUNO_JUNOCAM'}
        return look_up[super().instrument_id]
    
    @property
    def sensor_model_version(self):
        return 1

In [36]:
with juno_test(file, props={'kernels': updated_kernels}) as driver:
    print(driver.instrument_id)
    print(driver.ikid)
    print(driver.sensor_frame_id)
    print(driver.target_frame_id)
    print(to_isis(driver))

JUNOCAM


SpiceyError: Spice returns not found for function: bods2c